<a href="https://colab.research.google.com/github/EUNJIKIM810/pytorch-tutorial/blob/main/pytorch_tutorial_%EC%9E%90%EB%8F%99%EB%AF%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**torch.autograd를 사용한 자동 미분**


신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파입니다. 이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정됩니다.

이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있습니다. 이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원합니다.

입력 x, 매개변수 w와 b , 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다. PyTorch에서는 다음과 같이 정의할 수 있습니다:

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7c6eb95b5ae0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7c6eba122440>


변화도(Gradient) 계산하기

신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야 합니다. 즉, x와 y의 일부 고정값에서
∂
l
o
s
s
∂
w
∂w
∂loss
​
 와
∂
l
o
s
s
∂
b
∂b
∂loss
​
  가 필요합니다. 이러한 도함수를 계산하기 위해, loss.backward() 를 호출한 다음 w.grad와 b.grad에서 값을 가져옵니다:

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2758, 0.1156, 0.2678],
        [0.2758, 0.1156, 0.2678],
        [0.2758, 0.1156, 0.2678],
        [0.2758, 0.1156, 0.2678],
        [0.2758, 0.1156, 0.2678]])
tensor([0.2758, 0.1156, 0.2678])


변화도 추적 멈추기

기본적으로, requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다. 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우에는, 이러한 추적이나 지원이 필요 없을 수 있습니다. 연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있습니다:

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach() #텐서에 detach() 메소드를 사용
print(z_det.requires_grad)

False


변화도 추적을 멈춰야 하는 이유들은 다음과 같습니다:
신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시합니다.

변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상됩니다.

텐서 변화도와 야코비안 곱 (Jacobian Product)

야코비안 행렬 자체를 계산하는 대신, PyTorch는 주어진 입력 벡터
v
=
(
v
1
…
v
m
)
v=(v
1
​
 …v
m
​
 )에 대한 야코비안 곱(Jacobian Product)
v
T
⋅
J
v
T
 ⋅J을 계산합니다. 이 과정은
v
v를 인자로 backward를 호출하면 이뤄집니다.
v
v의 크기는 곱(product)을 계산하려고 하는 원래 텐서의 크기와 같아야 합니다.

In [6]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
